In [26]:
import numpy as np
def arbre_recomb(n,b):
    l=2*b*n+1
    S=np.zeros((l,n+1))
    for i in range(l):
        S[i,n]= b*n-i
    for j in range(1,n+1):
        for i in  range(l):
            if i>=b*j and i<=b*(2*n-j):
                S[i,n-j]=b*n-i
    return S

In [27]:
def proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n):
    proba=np.zeros((3,len(List_sigma)))
    h=T/n
    for i in range(len(List_sigma)):
        proba[0,i]=(List_sigma[i]**2+List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[1,i]=(List_sigma[i]**2-List_a[i]*Liste_saut[i]*sigma_bar*np.sqrt(h)+h*List_a[i]**2)/(2*(Liste_saut[i]*sigma_bar)**2)
        proba[2,i]=1-(proba[0,i]+proba[1,i])
    return proba

In [28]:
arbre=arbre_recomb(1000,2)

In [59]:
def prix_call_europeen(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    
    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [30]:
def transition_matrix(Q,h):
    nbr=Q.shape[0]
    tr_matrix=np.zeros((nbr,nbr))
    for i in range(nbr):
        for j in range(nbr):
            if i==j:
                tr_matrix[i,j]=np.exp(Q[i,j]*h)
            else:
                tr_matrix[i,j]=-(1-np.exp(Q[i,i]*h))*Q[i,j]/Q[i,i]
    return tr_matrix

In [31]:
Q=np.array([[-0.5,0.5],[0.5,-0.5]])
P=transition_matrix(Q,1/1000)

In [32]:
P

array([[9.99500125e-01, 4.99875021e-04],
       [4.99875021e-04, 9.99500125e-01]])

In [73]:
L1=[0.15,0.25]
L2=[0.05-0.5*0.15**2,0.05-0.5*0.25**2]
L3=[1,2]
L4=[0.05,0.05]

In [34]:
proba_up_middle_down(L1,L2,L3,0.2,1,1000)

array([[0.28433223, 0.19605476],
       [0.27820531, 0.19457244],
       [0.43746246, 0.6093728 ]])

In [35]:
A=prix_call_europeen(100,94,1000,2,0.2,1,L1,L2,L3,L4,P)

In [36]:
A[:,0][2000]
print(A[:,0][2000])

{0: 5.86121169713768, 1: 8.228404386030418}


In [37]:
def prix_put_europeen(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
        
    return table_prix

In [38]:
def prix_call_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(-K+S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],-K+S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

In [39]:
def prix_put_americain(K,S_o,n,b,sigma_bar,T,List_sigma,List_a,Liste_saut,List_r,P):
    proba_umd=proba_up_middle_down(List_sigma,List_a,Liste_saut,sigma_bar,T,n)
    l=2*b*n+1
    h=T/n
    table_prix=np.zeros((l,n+1),dtype=object)
    for i in range(l):
        table_prix[i,n]={reg:max(K-S_o*np.exp(arbre[i,n]*sigma_bar*np.sqrt(h)),0) for reg in range(len(List_sigma))}
    for j in range(1,n+1):
        for i in range(l):
            if i>=b*j and i<=b*(2*n-j):
                table_prix[i,n-j]={reg:0 for reg in range(len(List_sigma))}
                for etat in range(len(List_sigma)):
                    r=List_r[etat]
                    saut=Liste_saut[etat]
                    for k in range(len(List_sigma)):
                        table_prix[i,n-j][etat] += P[etat,k]*(table_prix[i-saut,n-j+1][k]*proba_umd[0,etat]+table_prix[i+saut,n-j+1][k]*proba_umd[1,etat]+table_prix[i,n-j+1][k]*proba_umd[2,etat])
                    table_prix[i,n-j][etat]=np.exp(-r*h)*table_prix[i,n-j][etat]
                    table_prix[i,n-j][etat]=max(table_prix[i,n-j][etat],K-S_o*np.exp(arbre[i,n-j]*sigma_bar*np.sqrt(h)))
        
    return table_prix

*Tests hyperparamètres*

In [76]:
K,n,T = 100, 1000, 1

List_sigma = np.array([0.15, 0.25]) #L1
Liste_saut = L3 #L3
List_r = np.array([0.05, 0.05]) #L4

List_d = np.array([0.04, 0.04])

# Calcul de List_a
List_a = List_r - List_d - 1/2 * List_sigma**2 #L2
L2 = List_a

In [92]:
SensitivySigmaTable0 = np.zeros((12, 8))
SensitivySigmaTable0[:, 0] = np.tile(np.array([90.0, 100.0, 110.0]), 4)

SensitivySigmaTable1 = np.zeros((12, 8))
SensitivySigmaTable1[:, 0] = np.tile(np.array([90.0, 100.0, 110.0]), 4)

[90.  0.  0.  0.  0.  0.]


In [90]:
def liste_saut(nbr_reg,liste_sigma,sigma_bar,liste_a):
    liste_saut = [0]*nbr_reg
    for i in range (nbr_reg):
        k1,k2 = np.floor(2*liste_sigma[i]/sigma_bar),np.ceil(2*liste_sigma[i]/sigma_bar)
        if k1==k2 or k1*sigma_bar<liste_sigma[i]:
            liste_saut[i] = k2
        else:
            c = ((k1*sigma_bar)**2-liste_sigma[i]**2)/liste_a[i]**2
            d = (k2*sigma_bar-np.sqrt((k2*sigma_bar)**2-4*liste_sigma[i]**2))**2/(4*liste_a[i])**2
            if c<=d:
                liste_saut[i] = k2
            else:
                liste_saut[i] = k1
    return liste_saut

In [88]:
ctes = [(0.1, 5), (0.15, 3), (0.2, 2), (0.25, 2), (0.3, 1)]

from tqdm import tqdm
import math

for j in tqdm(range(5)):
    print(j)
    arbre=arbre_recomb(n,ctes[j][1])
    
    for i in tqdm(range(3)):
        EC = prix_call_europeen(K, SensitivySigmaTable0[i, 0], n, ctes[j][1], ctes[j][0], T, L1, L2, [int(item) for item in liste_saut(2,L1,ctes[j][0],L2)], L4, P)[:,0][1000*ctes[j][1]]
        AC = prix_call_americain(K, SensitivySigmaTable0[i, 0], n, ctes[j][1], ctes[j][0], T, L1, L2, [int(item) for item in liste_saut(2,L1,ctes[j][0],L2)], L4, P)[:,0][1000*ctes[j][1]]
        EP = prix_put_europeen(K, SensitivySigmaTable0[i, 0], n, ctes[j][1], ctes[j][0], T, L1, L2, [int(item) for item in liste_saut(2,L1,ctes[j][0],L2)], L4, P)[:,0][1000*ctes[j][1]]
        AP = prix_call_europeen(K, SensitivySigmaTable0[i, 0], n, ctes[j][1], ctes[j][0], T, L1, L2, [int(item) for item in liste_saut(2,L1,ctes[j][0],L2)], L4, P)[:,0][1000*ctes[j][1]]
        SensitivySigmaTable0[i, j+1] = EC[0]
        SensitivySigmaTable0[3+i, j+1] = AC[0]
        SensitivySigmaTable0[6+i, j+1] = EP[0]
        SensitivySigmaTable0[9+i, j+1] = AP[0]
        SensitivySigmaTable1[i, j+1] = EC[1]
        SensitivySigmaTable1[3+i, j+1] = AC[1]
        SensitivySigmaTable1[6+i, j+1] = EP[1]
        SensitivySigmaTable1[9+i, j+1] = AP[1]
        
#Computing MD and RD
for i in tqdm(range(3)):
        SensitivySigmaTable0[i, -2] = SensitivySigmaTable0[i,1:-2]
        SensitivySigmaTable0[3+i, -2] = AC[0]
        SensitivySigmaTable0[6+i, -2] = EP[0]
        SensitivySigmaTable0[9+i, -2] = AP[0]

SyntaxError: incomplete input (1374591030.py, line 25)

In [ ]:
print(SensitivySigmaTable0)
print(SensitivySigmaTable1)

[[ 90.                  nan   4.04639237   4.04707864   4.04708268
    0.           0.           0.        ]
 [100.                  nan   9.33791883   9.33829962   9.33756193
    0.           0.           0.        ]
 [110.                  nan  16.81878718  16.81897125  16.81849481
    0.           0.           0.        ]
 [ 90.           0.           0.           0.           0.
    0.           0.           0.        ]
 [100.           0.           0.           0.           0.
    0.           0.           0.        ]
 [110.           0.           0.           0.           0.
    0.           0.           0.        ]
 [ 90.           0.           0.           0.           0.
    0.           0.           0.        ]
 [100.           0.           0.           0.           0.
    0.           0.           0.        ]
 [110.           0.           0.           0.           0.
    0.           0.           0.        ]
 [ 90.           0.           0.           0.           0.
    0.  